#  Downloading Audio
## Anna Bernbaum
## April 2019

In [1]:
import pandas as pd
import youtube_dl
from pydub import AudioSegment
from pydub.utils import which
from pydub.utils import mediainfo
import librosa as librosa
import os
from random import shuffle
import csv

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 5]

# Get all desired YouTubeIDs

Let's find the index and code associated with 'Cough':

In [2]:
cough_class_label_index = !grep Cough AudioSet/class_labels_indices.csv
print(cough_class_label_index)

speech_class_label_index = !grep Speech AudioSet/class_labels_indices.csv
print(speech_class_label_index)


['47,/m/01b_21,"Cough"']
['0,/m/09x0r,"Speech"', '7,/m/0brhx,"Speech synthesizer"']


Retreiving the class label:

In [3]:
print("Cough:", cough_class_label_index[0].split(",")[1])
print("Speech:", speech_class_label_index[0].split(",")[1])

Cough: /m/01b_21
Speech: /m/09x0r


Finding all samples with the selected label. The header of this table is printed.

In [4]:
# coughs = !grep /m/01b_21 AudioSet/balanced_train_segments.csv |head  # ID manually inserted
# print(type(coughs))
# print(coughs)

# Maybe select speech clips with only a speech tag
# ignore clips with speech and cough tag

coughs_bal = !grep /m/01b_21 AudioSet/balanced_train_segments.csv | cut -c -11  # get clip ID
coughs_bal_starts = !grep /m/01b_21 AudioSet/balanced_train_segments.csv | cut -d ',' -f 2  # get start time
print("Cough samples in Balanced Train:", len(coughs_bal))

speech_bal = !grep /m/09x0r AudioSet/balanced_train_segments.csv | cut -c -11
speech_bal_starts = !grep /m/09x0r AudioSet/balanced_train_segments.csv | cut -d ',' -f 2
print("Speech samples in Balanced Train:", len(speech_bal))


coughs_eval = !grep /m/01b_21 AudioSet/eval_segments.csv | cut -c -11
coughs_eval_starts = !grep /m/01b_21 AudioSet/eval_segments.csv | cut -d ',' -f 2
print("\nCough samples in Evaluation:", len(coughs_eval))

speech_eval = !grep /m/09x0r AudioSet/eval_segments.csv | cut -c -11
speech_eval_starts = !grep /m/09x0r AudioSet/eval_segments.csv | cut -d ',' -f 2
print("Speech samples in Evaluation:", len(speech_eval))


coughs_unbal = !grep /m/01b_21 AudioSet/unbalanced_train_segments.csv | cut -c -11
coughs_unbal_starts = !grep /m/01b_21 AudioSet/unbalanced_train_segments.csv | cut -d ',' -f 2
print("\nCough samples in Unbalanced Train:", len(coughs_unbal))

speech_unbal = !grep /m/09x0r AudioSet/unbalanced_train_segments.csv | cut -c -11
speech_unbal_starts = !grep /m/09x0r AudioSet/unbalanced_train_segments.csv | cut -d ',' -f 2
print("Speech samples in Unbalanced Train:", len(speech_unbal))
      
print("\nTotal Number of Cough Clips:", (len(coughs_bal)+len(coughs_eval)+len(coughs_unbal)))
print("\nTotal Number of Speech Clips:", (len(speech_bal)+len(speech_eval)+len(speech_unbal)))

Cough samples in Balanced Train: 60
Speech samples in Balanced Train: 5735

Cough samples in Evaluation: 60
Speech samples in Evaluation: 5324

Cough samples in Unbalanced Train: 751
Speech samples in Unbalanced Train: 999421

Total Number of Cough Clips: 871

Total Number of Speech Clips: 1010480


It may be a problem that some clips contain more than one sound tag. Let's investigate how big the dataset would be if various options were removed:

In [5]:
# How many clips are labelled with only cough or speech?
# For each cough tag, count the number of labels

i = 0
j = 0
k = 0

l = 0
m = 0
n = 0

coughs_bal = []  # clip IDs
coughs_bal_starts = []  # get start time
speech_bal = []
speech_bal_starts = []

with open("AudioSet/balanced_train_segments.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        length = (len(row))
        if length < 5:  # if there is only one tag
            if any('/m/01b_21' in s for s in row):  # if cough
                i = i + 1
                j = j + 1
                k = k + 1
                coughs_bal.append(row[0])
                coughs_bal_starts.append(row[1])
            if any('/m/09x0r' in s for s in row):  # if speech
                l = l + 1
                m = m + 1
                n = n + 1
                speech_bal.append(row[0])
                speech_bal_starts.append(row[1])
        
        elif length < 6: # if there are only 2 tags
            if any('/m/01b_21' in s for s in row):
                j = j + 1
                k = k + 1
            if any('/m/09x0r' in s for s in row):
                m = m + 1
                n = n + 1


        elif length < 7: # if there are only 2 tags
            if any('/m/01b_21' in s for s in row):
                k = k + 1
            if any('/m/09x0r' in s for s in row):
                n = n + 1
        
        else:
            pass
        
print('There are ', i, 'cough clips with 1 tag in the balanced train dataset')
print('There are ', j, 'cough clips with 2 tags in the balanced train dataset')
print('There are ', k, 'cough clips with 3 tags in the balanced train dataset')
print('\nThere are ', l, 'speech clips with 1 tag in the balanced train dataset')
print('There are ', m, 'speech clips with 2 tags in the balanced train dataset')
print('There are ', n, 'speech clips with 3 tags in the balanced train dataset')



i = 0
j = 0
k = 0

l = 0
m = 0
n = 0

coughs_unbal = []  # clip IDs
coughs_unbal_starts = []  # get start time
speech_unbal = []
speech_unbal_starts = []

with open("AudioSet/unbalanced_train_segments.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        length = (len(row))
        if length < 5:  # if there is only one tag
            if any('/m/01b_21' in s for s in row):  # if cough
                i = i + 1
                j = j + 1
                k = k + 1
                coughs_unbal.append(row[0])
                coughs_unbal_starts.append(row[1])
            if any('/m/09x0r' in s for s in row):  # if speech
                l = l + 1
                m = m + 1
                n = n + 1
                speech_unbal.append(row[0])
                speech_unbal_starts.append(row[1])
        
        
        elif length < 6: # if there are only 2 tags
            if any('/m/01b_21' in s for s in row):
                j = j + 1
                k = k + 1
            if any('/m/09x0r' in s for s in row):
                m = m + 1
                n = n + 1


        elif length < 7: # if there are only 2 tags
            if any('/m/01b_21' in s for s in row):
                k = k + 1
            if any('/m/09x0r' in s for s in row):
                n = n + 1
        
        else:
            pass
        
print('\n\nThere are ', i, 'cough clips with 1 tag in the unbalanced train dataset')
print('There are ', j, 'cough clips with 2 tags in the unbalanced train dataset')
print('There are ', k, 'cough clips with 3 tags in the unbalanced train dataset')
print('\nThere are ', l, 'speech clips with 1 tag in the unbalanced train dataset')
print('There are ', m, 'speech clips with 2 tags in the unbalanced train dataset')
print('There are ', n, 'speech clips with 3 tags in the unbalanced train dataset')

i = 0
j = 0
k = 0

l = 0
m = 0
n = 0

coughs_eval = []  # clip IDs
coughs_eval_starts = []  # get start time
speech_eval = []
speech_eval_starts = []

with open("AudioSet/eval_segments.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        length = (len(row))
        if length < 5:  # if there is only one tag
            if any('/m/01b_21' in s for s in row):  # if cough
                i = i + 1
                j = j + 1
                k = k + 1
                coughs_eval.append(row[0])
                coughs_eval_starts.append(row[1])
            if any('/m/09x0r' in s for s in row):  # if speech
                l = l + 1
                m = m + 1
                n = n + 1
                speech_eval.append(row[0])
                speech_eval_starts.append(row[1])
        
        
        elif length < 6: # if there are only 2 tags
            if any('/m/01b_21' in s for s in row):
                j = j + 1
                k = k + 1
            if any('/m/09x0r' in s for s in row):
                m = m + 1
                n = n + 1


        elif length < 7: # if there are only 2 tags
            if any('/m/01b_21' in s for s in row):
                k = k + 1
            if any('/m/09x0r' in s for s in row):
                n = n + 1
        
        else:
            pass
        
print('\n\nThere are ', i, 'cough clips with 1 tag in the evaluation dataset')
print('There are ', j, 'cough clips with 2 tags in the evaluation dataset')
print('There are ', k, 'cough clips with 3 tags in the evaluation dataset')
print('\nThere are ', l, 'speech clips with 1 tag in the evaluation dataset')
print('There are ', m, 'speech clips with 2 tags in the evaluation dataset')
print('There are ', n, 'speech clips with 3 tags in the evaluation dataset')

There are  15 cough clips with 1 tag in the balanced train dataset
There are  37 cough clips with 2 tags in the balanced train dataset
There are  48 cough clips with 3 tags in the balanced train dataset

There are  0 speech clips with 1 tag in the balanced train dataset
There are  2474 speech clips with 2 tags in the balanced train dataset
There are  4608 speech clips with 3 tags in the balanced train dataset


There are  239 cough clips with 1 tag in the unbalanced train dataset
There are  525 cough clips with 2 tags in the unbalanced train dataset
There are  677 cough clips with 3 tags in the unbalanced train dataset

There are  440192 speech clips with 1 tag in the unbalanced train dataset
There are  819642 speech clips with 2 tags in the unbalanced train dataset
There are  933763 speech clips with 3 tags in the unbalanced train dataset


There are  18 cough clips with 1 tag in the evaluation dataset
There are  39 cough clips with 2 tags in the evaluation dataset
There are  52 cough

Let's create our own Train : Evaluation : Validation split to maximise the number of clips used. This will be done in a separate file, but here we combine all available clips.

In [6]:
# combine the lists of youtube ids into one master list
all_coughs = (coughs_bal + coughs_eval + coughs_unbal)
all_speech = (speech_bal + speech_eval + speech_unbal)

all_coughs_starts = (coughs_bal_starts + coughs_eval_starts + coughs_unbal_starts)
all_speech_starts = (speech_bal_starts + speech_eval_starts + speech_unbal_starts)


# create pandas dataframe of YouTubeIDs
speech_df = pd.DataFrame()  # create empty dataframe
cough_df = pd.DataFrame()
speech_df['Speech YouTubeIDs'] = pd.Series(all_speech)
cough_df['Coughs YouTubeIDs'] = pd.Series(all_coughs)
speech_df['Speech Starts'] = pd.Series(all_speech_starts)
cough_df['Coughs Starts'] = pd.Series(all_coughs_starts)

# Shuffle the dataframes
speech_df = speech_df.sample(frac=1).reset_index(drop=True)
cough_df = cough_df.sample(frac=1).reset_index(drop=True)

# Trim Speech Df
max_len = cough_df.shape[0]
speech_df = speech_df[:max_len]

print(speech_df)
print(cough_df)

    Speech YouTubeIDs Speech Starts
0         AQgr0ABLLCU        30.000
1         Nf5ceLnwXVk        30.000
2         IFrgnM2hBDM        30.000
3         0oLuOoKzIbI        30.000
4         d_nWeecVBc4        30.000
5         _a-8OqLH-cw        30.000
6         U8Lwl5iIsQw       420.000
7         cAYYTzKUtTM        30.000
8         1vDnqREJkok        30.000
9         9DM7cn5H3_0        30.000
10        0UeNO8yD9uc         0.000
11        Z2x2cgW3NFU        30.000
12        FdS6RdPpnJ8        70.000
13        r8WrdJm_hGU        30.000
14        QciztgPcgVE        30.000
15        B9xezYosg-c        30.000
16        Lva1QDgious        30.000
17        -NF063PVavc        30.000
18        J1Zj36ds9xw        30.000
19        XtlkzJZt7Gs        30.000
20        E6qZ8RLQrgI        30.000
21        12SYZowiUvc        30.000
22        WDGQdxhpsGI        30.000
23        AC2BK3hQVWc        30.000
24        B4dvoudOFpU        30.000
25        V3tZw1NQ-VI        30.000
26        L1BNBkO8Mpg       

# Find Associated YouTube clip for each YouTubeID

https://stackoverflow.com/questions/27473526/download-only-audio-from-youtube-video-using-youtube-dl-in-python-script

https://stackoverflow.com/questions/28423501/download-part-of-the-youtube-video-using-python


## Getting all Speech Clips

In [7]:
speech_test_df = speech_df[:300]

speech_fail_log = []

for i in range(len(speech_test_df.index)):
    title = speech_test_df['Speech YouTubeIDs'][i] + '.wav'  # filename
    url = "http://www.youtube.com/watch?v=" + title  # URL

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'Untrimmed_AudioSet_WAV_files/' + title,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # WAV file
            'preferredquality': '192',
        }],
    }
    
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Find the associated start_time
        start_time = float(speech_test_df['Speech Starts'][i])

        # Trim the clip
        y, sr = librosa.load('Untrimmed_AudioSet_WAV_files/'+ title, offset=start_time, duration=10.0) # trim a 10 second segment from start_time    

    #     # plot the clip - used for debugging
    #     plt.title(title)
    #     plt.xlabel('sample number')
    #     plt.ylabel('Amplitude')
    #     plt.plot(y)
    #     plt.show()

        # write a wav file of the trimmed clip
        filepath = 'Audioset_WAV_files/Speech/' + title
        librosa.output.write_wav(filepath, y, sr, norm=True)
        librosa.output.write_wav(filepath, y, sr, norm=True)

        
    # if the clip is unavailable
    except: 
        speech_fail_log.append(speech_test_df['Speech YouTubeIDs'][i])

        
print(speech_fail_log)

[youtube] AQgr0ABLLCU: Downloading webpage
[youtube] AQgr0ABLLCU: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/AQgr0ABLLCU.wav
[download] 100% of 704.80KiB in 00:0085MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/AQgr0ABLLCU.wav exists, skipping
[youtube] Nf5ceLnwXVk: Downloading webpage
[youtube] Nf5ceLnwXVk: Downloading video info webpage
[youtube] Nf5ceLnwXVk: Downloading MPD manifest
[dashsegments] Total fragments: 85
[download] Destination: Untrimmed_AudioSet_WAV_files/Nf5ceLnwXVk.wav
[download] 100% of 12.17MiB in 00:33.56MiB/s ETA 00:00080
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/Nf5ceLnwXVk.wav exists, skipping
[youtube] IFrgnM2hBDM: Downloading webpage
[youtube] IFrgnM2hBDM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/IFrgnM2hBDM.wav
[download] 100% of 8.47MiB in 00:0266MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/IFrgnM2hBD

ERROR: This video contains content from Victor Entertainment, Inc., who has blocked it on copyright grounds.


[youtube] cAYYTzKUtTM: Downloading webpage
[youtube] cAYYTzKUtTM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/cAYYTzKUtTM.wav
[download] 100% of 12.66MiB in 00:0385MiB/s ETA 00:004
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/cAYYTzKUtTM.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/cAYYTzKUtTM.wav exists, skipping
[youtube] 1vDnqREJkok: Downloading webpage
[youtube] 1vDnqREJkok: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/1vDnqREJkok.wav
[download] 100% of 960.27KiB in 00:0199MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/1vDnqREJkok.wav exists, skipping
[youtube] 9DM7cn5H3_0: Downloading webpage
[youtube] 9DM7cn5H3_0: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/9DM7cn5H3_0.wav
[download] 100% of 4.59MiB in 00:0198MiB/s ETA 00:008
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/9DM7cn5H3_0.wav exists,

ERROR: This video is unavailable.


[youtube] Z2x2cgW3NFU: Downloading webpage
[youtube] Z2x2cgW3NFU: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/Z2x2cgW3NFU.wav
[download] 100% of 9.12MiB in 00:0201MiB/s ETA 00:004
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/Z2x2cgW3NFU.wav exists, skipping
[youtube] FdS6RdPpnJ8: Downloading webpage
[youtube] FdS6RdPpnJ8: Downloading video info webpage
[youtube] FdS6RdPpnJ8: Downloading MPD manifest
[dashsegments] Total fragments: 9
[download] Destination: Untrimmed_AudioSet_WAV_files/FdS6RdPpnJ8.wav
[download] 100% of 1.30MiB in 00:03.38MiB/s ETA 00:00017
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/FdS6RdPpnJ8.wav exists, skipping
[youtube] r8WrdJm_hGU: Downloading webpage
[youtube] r8WrdJm_hGU: Downloading video info webpage


...

[youtube] 3zXoWTrcSm4: Downloading webpage
[youtube] 3zXoWTrcSm4: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/3zXoWTrcSm4.wav
[download] 100% of 1.73MiB in 00:0175MiB/s ETA 00:002
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/3zXoWTrcSm4.wav exists, skipping
[youtube] BWKLHEzJF04: Downloading webpage
[youtube] BWKLHEzJF04: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/BWKLHEzJF04.wav
[download] 100% of 67.89KiB in 00:0036MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/BWKLHEzJF04.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/BWKLHEzJF04.wav exists, skipping
[youtube] aUEi1LmR7pI: Downloading webpage
[youtube] aUEi1LmR7pI: Downloading video info webpage
[youtube] aUEi1LmR7pI: Downloading MPD manifest
[dashsegments] Total fragments: 10
[download] Destination: Untrimmed_AudioSet_WAV_files/aUEi1LmR7pI.wav
[download] 100% of 1.41MiB in 00:04.14MiB/s ETA 00:0

ERROR: This video is unavailable.


[youtube] _5AxeccpKWc: Downloading webpage
[youtube] _5AxeccpKWc: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/_5AxeccpKWc.wav
[download] 100% of 1.47MiB in 00:0040MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/_5AxeccpKWc.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/_5AxeccpKWc.wav exists, skipping
[youtube] 1dpSxxzpkRs: Downloading webpage
[youtube] 1dpSxxzpkRs: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/1dpSxxzpkRs.wav
[download] 100% of 1.18MiB in 00:0005MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/1dpSxxzpkRs.wav exists, skipping
[youtube] o2s1u7goQ0E: Downloading webpage
[youtube] o2s1u7goQ0E: Downloading video info webpage


ERROR: This video is no longer available because the YouTube account associated with this video has been terminated.


[youtube] cdeKJCvTFvM: Downloading webpage
[youtube] cdeKJCvTFvM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/cdeKJCvTFvM.wav
[download] 100% of 190.66KiB in 00:0096MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/cdeKJCvTFvM.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/cdeKJCvTFvM.wav exists, skipping
[youtube] -XvR4PkzMPA: Downloading webpage
[youtube] -XvR4PkzMPA: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/-XvR4PkzMPA.wav
[download] 100% of 3.17MiB in 00:0027MiB/s ETA 00:003
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/-XvR4PkzMPA.wav exists, skipping
[youtube] tcHZgaRw1uM: Downloading webpage
[youtube] tcHZgaRw1uM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/tcHZgaRw1uM.wav
[download] 100% of 1.22MiB in 00:0165MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/tcHZgaRw1uM.wav exists, 

## Getting all cough clips

In [8]:
coughs_test_df = cough_df[:300]

coughs_fail_log = []

for i in range(len(coughs_test_df.index)):
    title = coughs_test_df['Coughs YouTubeIDs'][i] + '.wav'  # filename
    url = "http://www.youtube.com/watch?v=" + title  # URL

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'Untrimmed_AudioSet_WAV_files/' + title,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # WAV file
            'preferredquality': '192',
        }],
    }
    
    # if clip is unavailable
    # if clip is longer than X skip
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        # Find the associated start_time
        start_time = float(coughs_test_df['Coughs Starts'][i])


        # Trim the clip
        y, sr = librosa.load('Untrimmed_AudioSet_WAV_files/' + title, offset=start_time, duration=10.0) # trim a 10 second segment from start_time    

    #     # plot the clip - used for debugging
    #     plt.title(title)
    #     plt.xlabel('sample number')
    #     plt.ylabel('Amplitude')
    #     plt.plot(y)
    #     plt.show()

        # write a wav file of the trimmed clip
        filepath = 'Audioset_WAV_files/Coughs/' + title
        librosa.output.write_wav(filepath, y, sr, norm=True)
        librosa.output.write_wav(filepath, y, sr, norm=True)

    # if clip is unavailable
    except: 
        coughs_fail_log.append(coughs_test_df['Coughs YouTubeIDs'][i])
        # create an error log

print(coughs_fail_log)

[youtube] sBQmYKx44vw: Downloading webpage
[youtube] sBQmYKx44vw: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/sBQmYKx44vw.wav
[download] 100% of 1.81MiB in 00:0018MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/sBQmYKx44vw.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/sBQmYKx44vw.wav exists, skipping
[youtube] rllrJN6IFJs: Downloading webpage
[youtube] rllrJN6IFJs: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/rllrJN6IFJs.wav has already been downloaded
[download] 100% of 4.97MiB
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/rllrJN6IFJs.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/rllrJN6IFJs.wav exists, skipping
[youtube] 4EiskLYtkO4: Downloading webpage
[youtube] 4EiskLYtkO4: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/4EiskLYtkO4.wav
[download] 100% of 153.54KiB in 00:0174MiB/s ETA 00:000
[ffmpeg] Cor

ERROR: This video is no longer available because the YouTube account associated with this video has been terminated.


[youtube] BqK5GsVPpB8: Downloading webpage
[youtube] BqK5GsVPpB8: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/BqK5GsVPpB8.wav
[download] 100% of 221.22KiB in 00:0036MiB/s ETA 00:000
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/BqK5GsVPpB8.wav exists, skipping
[youtube] aTHVNwrEVxI: Downloading webpage
[youtube] aTHVNwrEVxI: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/aTHVNwrEVxI.wav has already been downloaded
[download] 100% of 2.28MiB
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/aTHVNwrEVxI.wav exists, skipping
[youtube] GTiou-fTfQs: Downloading webpage
[youtube] GTiou-fTfQs: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/GTiou-fTfQs.wav has already been downloaded
[download] 100% of 981.16KiB
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/GTiou-fTfQs.wav exists, skipping
[youtube] K7k1tH72E4k: Downloading webpage
[youtube] K7k1tH72E4k: Downloading video info webp

ERROR: This video is unavailable.


[youtube] KCqugHUa6lY: Downloading webpage
[youtube] KCqugHUa6lY: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/KCqugHUa6lY.wav
[download] 100% of 707.24KiB in 00:0035MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/KCqugHUa6lY.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/KCqugHUa6lY.wav exists, skipping
[youtube] YtdWQYALUHI: Downloading webpage
[youtube] YtdWQYALUHI: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/YtdWQYALUHI.wav has already been downloaded
[download] 100% of 324.59KiB
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/YtdWQYALUHI.wav exists, skipping
[youtube] gCI4upmqTg8: Downloading webpage
[youtube] gCI4upmqTg8: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/gCI4upmqTg8.wav
[download] 100% of 879.19KiB in 00:0070MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/gCI4upmqTg8.wav exists, skipping

...

[youtube] jATCv2v-iXo: Downloading webpage
[youtube] jATCv2v-iXo: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/jATCv2v-iXo.wav
[download] 100% of 1.17MiB in 00:0043MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/jATCv2v-iXo.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/jATCv2v-iXo.wav exists, skipping
[youtube] EmAi3m5j0QQ: Downloading webpage
[youtube] EmAi3m5j0QQ: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/EmAi3m5j0QQ.wav has already been downloaded
[download] 100% of 4.54MiB
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/EmAi3m5j0QQ.wav exists, skipping
[youtube] oVE7t_RU8V8: Downloading webpage
[youtube] oVE7t_RU8V8: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/oVE7t_RU8V8.wav
[download] 100% of 1.12MiB in 00:0037MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/oVE7t_RU8V8.wav exists, skipping
[yout

ERROR: This video is unavailable.


[youtube] B4dHtSx7OGc: Downloading webpage
[youtube] B4dHtSx7OGc: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/B4dHtSx7OGc.wav has already been downloaded
[download] 100% of 1.95MiB
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/B4dHtSx7OGc.wav exists, skipping
[youtube] EfYf7arPeNk: Downloading webpage
[youtube] EfYf7arPeNk: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/EfYf7arPeNk.wav
[download] 100% of 1.93MiB in 00:0056MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/EfYf7arPeNk.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/EfYf7arPeNk.wav exists, skipping
[youtube] -UxFUu6iiiY: Downloading webpage
[youtube] -UxFUu6iiiY: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/-UxFUu6iiiY.wav has already been downloaded
[download] 100% of 5.61MiB
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/-UxFUu6iiiY.wav"
[ffmpeg] Post-process file

ERROR: This video is unavailable.


[youtube] BwfscnYTNnA: Downloading webpage
[youtube] BwfscnYTNnA: Downloading video info webpage
[download] Untrimmed_AudioSet_WAV_files/BwfscnYTNnA.wav has already been downloaded
[download] 100% of 224.68KiB
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/BwfscnYTNnA.wav exists, skipping
[youtube] b3P2awI0fts: Downloading webpage
[youtube] b3P2awI0fts: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/b3P2awI0fts.wav
[download] 100% of 731.26KiB in 00:0056MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/b3P2awI0fts.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/b3P2awI0fts.wav exists, skipping
[youtube] I8q1M4pl0mY: Downloading webpage
[youtube] I8q1M4pl0mY: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/I8q1M4pl0mY.wav
[download] 100% of 959.60KiB in 00:0048MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/I8q1M4pl0mY.wav"
[ffmpeg]

## Check number of clips after downloading
Some clips were lost due to failed downloads. The two datagroups need to be evened out.

In [9]:
# Check number of clips in each folder
coughs_count=0
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_count+=1
print("Unedited number of cough clips:", coughs_count)


speech_count=0
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_count+=1
print("Unedited number of speech clips:", speech_count)

# Remove additional clips to even out the size
values = {"Coughs": coughs_count, "Speech": speech_count}
largest = max(values, key=values.get)  # find which group has more clips
difference = (abs(coughs_count - speech_count))  # how many clips should be removed

# find names of clips to be removed
f = []
for (dirpath, dirnames, filenames) in os.walk("Audioset_WAV_files/"+ largest + '/'):
    f.extend(filenames)
    break

shuffle(f)

clips_to_remove = f[:difference]
print(clips_to_remove)

for clip in clips_to_remove:
    os.remove("Audioset_WAV_files/"+ largest + '/' + clip)

# Check it has worked
coughs_count=0
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_count+=1
print("Unedited number of cough clips:", coughs_count)


speech_count=0
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_count+=1
print("Unedited number of speech clips:", speech_count)

Unedited number of cough clips: 223
Unedited number of speech clips: 222
['1j1duoxdxBg.wav']
Unedited number of cough clips: 222
Unedited number of speech clips: 222


# Save YouTube IDs of final clip selections

In [10]:
# get filename of every file in both directories
# write to CSV  selected_YouTubeIDs

coughs_clips = ["Coughs"]
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_clips.append(files)

speech_clips = ["Speech"]
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_clips.append(files)

csvData = zip(coughs_clips, speech_clips)      
with open('selected_YouTubeIDs_onetag.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)